In [ ]:
import getpass
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from titlecase import titlecase

In [ ]:
# Read and parse the source spreadsheet

from openpyxl import load_workbook
wb = load_workbook(filename='Organizations_module.xlsx')
ws = wb['output']

source_data = list()
for num, row in enumerate(ws.rows):
    if num == 0:
        headers = [i.value for i in row]
    else:
        source_data.append(dict(zip(headers, [i.value for i in row])))

In [ ]:
def format_publishers(source_data):
    for row_dict in source_data:
        row_dict['formatted_publisher'] = titlecase(row_dict['Publisher'])
format_publishers(source_data)

In [ ]:
driver = webdriver.Firefox()
def open_site():
    driver.get("https://lsu-erm.bc.sirsidynix.net/auth/?service=/organizations/")

open_site()

In [ ]:
# Split into those not yet found in the system and those found

def open_resultspage_by_partial_word(partial_word):
    input_box = driver.find_element_by_id('searchOrganizationName')
    input_box.clear()
    input_box.send_keys(partial_word)
    input_box.send_keys(Keys.RETURN);

def org_in_dataTable(org):
    try:
        truncated_org = org[:15]
        link = driver.find_element_by_xpath(f'//table[@class="dataTable"]/tbody/tr/td/a[contains(text(), "{truncated_org}")]')
        if link.text.strip() == org.strip():
            return link
        else:
            return False
    except NoSuchElementException:
        # use false to indicate no recorded org with that title
        return False

founds = list()
not_founds = list()

for item in source_data:
    organization = item['formatted_publisher']
    open_resultspage_by_partial_word(organization)
    time.sleep(1)
    match = org_in_dataTable(organization)
    if match:
        founds.append(item)
    else:
        not_founds.append(item)

In [ ]:
def go_to_homepage():
    driver.get("https://lsu-erm.bc.sirsidynix.net/organizations/")

def enter_org_page(org):
    link = driver.find_element_by_xpath(f'//table[@class="dataTable"]/tbody/tr/td/a[text()="{org}"]')
    link.click()
    
def make_new_org(item):
    new_org_link = driver.find_element_by_xpath('//div[@class="main-menu-link"]/span[contains(text(), "New Organization")]')
    new_org_link.click()

def edit_existing_org():
    existing_org_link = driver.find_element_by_xpath('//a[@id="editOrganization"]')
    existing_org_link.click()
    
def add_org_info(item):
    name_field = driver.find_element_by_xpath('//input[@id="organizationName"]')
    name_field.clear()
    name_field.send_keys(item['formatted_publisher'])
    parent_field = driver.find_element_by_id("parentOrganization")
    parent_field.clear()
    parent_field.send_keys(item['parent_name'] or '')
    url_field = driver.find_element_by_xpath('//input[@id="companyURL"]')
    url_field.clear()
    url_field.send_keys(item['url'])
    publisher_toggle = driver.find_element_by_xpath('//input[@id="5"]')
    publisher_toggle.click()
    vendor_toggle = driver.find_element_by_xpath('//input[@id="6"]')
    vendor_toggle.click()
    submit_button = driver.find_element_by_xpath('//input[@id="submitOrganizationChanges"]')
    submit_button.click()
    
def add_contact_info(item):
    contact_button = driver.find_element_by_xpath('//a[@class="showContacts"]')
    contact_button.click()
    time.sleep(2)
    add_contact_link = driver.find_element_by_xpath('//a[contains(text(), "add contact")]')
    add_contact_link.click()
    time.sleep(2)
    name_field = driver.find_element_by_id("contactName")
    name_field.clear()
    name_field.send_keys('General Contact')
    email_field = driver.find_element_by_xpath('//input[@id="emailAddress"]')
    email_field.clear()
    email_field.send_keys(item['contact_email'])
    phone_field = driver.find_element_by_xpath('//input[@id="phoneNumber"]')
    phone_field.clear()
    phone_field.send_keys(item['contact_phone'])
    support_toggle = driver.find_element_by_xpath('//input[@id="4"]')
    support_toggle.click()
    notes_field = driver.find_element_by_xpath('//textarea[@id="noteText"]')
    notes_field.clear()
    notes_field.send_keys("Content provided by EBSCO, contact EBSCO for support issues")
    submit_button = driver.find_element_by_xpath('//input[@id="submitContactForm"]')
    submit_button.click()

In [ ]:
def make_a_found(item):
    go_to_homepage()
    time.sleep(1)
    open_resultspage_by_partial_word(item['formatted_publisher'][:15])
    time.sleep(1)
    enter_org_page(item['formatted_publisher'])
    time.sleep(1)
    edit_existing_org()
    time.sleep(1)
    add_org_info(item)
    time.sleep(3)
    add_contact_info(item)
    time.sleep(1)

In [ ]:
def make_a_notfound(item):
    go_to_homepage()
    time.sleep(1)
    open_resultspage_by_partial_word(item['formatted_publisher'][:15])
    time.sleep(1)
    make_new_org(item)
    time.sleep(1)
    add_org_info(item)
    time.sleep(6)
    add_contact_info(item)
    time.sleep(1)

In [ ]:
# actually do the damage to founds
# for item in founds:
#     print(item['Publisher'])
#     make_a_found(item)

In [ ]:
# actually do the damage to not_founds
# for item in not_founds:
#     print(item['Publisher'])
#     make_a_notfound(item)

In [ ]:
# if you just want to try making one - for experiment

sample_item = {'Publisher': 'A FAKE ITEM', 'Vendor': 'Fake vendor', 'provider': 'Fake', 'url': 'https://example.com', 'parent_name': 'Fake Parent', 'contact_email': 'fake@example.com', 'contact_phone': '867-5209', 'formatted_publisher': 'A FAKE ITEM'}
make_a_found(sample_item)